# **ETL with Python in the Colab environment**
### **Created by:** Cristina
### **Created on:** 10/09/2025
### **Description:** refresher ETL with Python: how to load data into a pandas data frame from csv, and clean the data, then export it to Tableau

### **Challenges:** file is too big to open as raw in github to get the link.

####Actually, this was an excel file, and once converted to csv, issue was solved.

###Two columns disapeared from the clean df (postal_code and cleaned_postal_code), rerun all the cells, and now they came back.

###Exporting from data frame to csv within the Colab environment takes it to the working dir, which is temporary, and it took me a bit to figure that out.

### **Reminder** to remove the access once I am done with the project.
### **Note:** data cleaning: clean the postal codes, as some were missing leading zeros
### **Note:** For more heavy duty querying, use sqlalchemy. Will upgrade to that in the next version.
### **Data Sources**:

### **To be continued**

In [ ]:
import pandas as pd
import sqlite3
import glob
import matplotlib.pyplot as plt






In [ ]:
path = 'https://raw.githubusercontent.com/cristina0000/2025_git_projects/refs/heads/main/p5_tableau_superstore/data/sample_superstore.csv'


sales_df=pd.read_csv(path)
sales_df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [ ]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

## Transforming data with Python

In [ ]:
#clean headers (replace blanks with underscores):
sales_df.columns = sales_df.columns.str.lower().str.replace(" ", "_")
sales_df.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,...,postal_code,region,product_id,category,sub-category,product_name,sales,quantity,discount,profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [ ]:
sales_df.drop_duplicates()
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   row_id         9994 non-null   int64  
 1   order_id       9994 non-null   object 
 2   order_date     9994 non-null   object 
 3   ship_date      9994 non-null   object 
 4   ship_mode      9994 non-null   object 
 5   customer_id    9994 non-null   object 
 6   customer_name  9994 non-null   object 
 7   segment        9994 non-null   object 
 8   country        9994 non-null   object 
 9   city           9994 non-null   object 
 10  state          9994 non-null   object 
 11  postal_code    9994 non-null   int64  
 12  region         9994 non-null   object 
 13  product_id     9994 non-null   object 
 14  category       9994 non-null   object 
 15  sub-category   9994 non-null   object 
 16  product_name   9994 non-null   object 
 17  sales          9994 non-null   float64
 18  quantity

In [ ]:
sales_df.dropna()
sales_df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   row_id         9994 non-null   int64  
 1   order_id       9994 non-null   object 
 2   order_date     9994 non-null   object 
 3   ship_date      9994 non-null   object 
 4   ship_mode      9994 non-null   object 
 5   customer_id    9994 non-null   object 
 6   customer_name  9994 non-null   object 
 7   segment        9994 non-null   object 
 8   country        9994 non-null   object 
 9   city           9994 non-null   object 
 10  state          9994 non-null   object 
 11  postal_code    9994 non-null   int64  
 12  region         9994 non-null   object 
 13  product_id     9994 non-null   object 
 14  category       9994 non-null   object 
 15  sub-category   9994 non-null   object 
 16  product_name   9994 non-null   object 
 17  sales          9994 non-null   float64
 18  quantity

In [ ]:
#open a database connection (sqlite)
#this creates an empty sqlite database file:
#con = sqlite3.connect("sales.sqlite")

conn = sqlite3.connect("salesdb.sqlite")
#write the data frame to a sqlite table:
sales_df.to_sql('salesdb_table', conn, if_exists = 'replace', index = False) # could also do append
conn.commit()

In [ ]:

# get a cursor in order to execute SQL statements and fetch results:
cur = conn.cursor()

#check that the table was created
res = cur.execute ("SELECT name FROM sqlite_schema")
#this does the same as above
#res = cur.execute ("SELECT name FROM sqlite_master")
res.fetchall()
#print (res)

#commit after inserting:
#con.commit()





[('salesdb_table',)]

In [ ]:
#execute a SQL query on the table salesdb_table:
result = cur.execute ("SELECT * FROM salesdb_table")
#result
#result.fetchall()
result.fetchone()







(1,
 'CA-2016-152156',
 '11/8/2016',
 '11/11/2016',
 'Second Class',
 'CG-12520',
 'Claire Gute',
 'Consumer',
 'United States',
 'Henderson',
 'Kentucky',
 42420,
 'South',
 'FUR-BO-10001798',
 'Furniture',
 'Bookcases',
 'Bush Somerset Collection Bookcase',
 261.96,
 2,
 0.0,
 41.9136)

In [ ]:
#checking for missing leading zeros in the postal code or more than 5 digits
result = cur.execute ('''SELECT state, city, postal_code, LENGTH(postal_code) as postal_code_length
                          FROM salesdb_table
                          WHERE LENGTH(postal_code) > 5 OR LENGTH(postal_code) < 5
                          ORDER BY LENGTH(postal_code) DESC
                      ''')
result.fetchall()

[('Connecticut', 'Fairfield', 6824, 4),
 ('New Jersey', 'Westfield', 7090, 4),
 ('New Jersey', 'Morristown', 7960, 4),
 ('New Jersey', 'Belleville', 7109, 4),
 ('New Jersey', 'Belleville', 7109, 4),
 ('New Jersey', 'Belleville', 7109, 4),
 ('New Jersey', 'Belleville', 7109, 4),
 ('New Jersey', 'Belleville', 7109, 4),
 ('New Jersey', 'Lakewood', 8701, 4),
 ('New Jersey', 'Lakewood', 8701, 4),
 ('New Jersey', 'Hackensack', 7601, 4),
 ('Massachusetts', 'Lowell', 1852, 4),
 ('Massachusetts', 'Lowell', 1852, 4),
 ('Massachusetts', 'Lowell', 1852, 4),
 ('Connecticut', 'Manchester', 6040, 4),
 ('Connecticut', 'Manchester', 6040, 4),
 ('Connecticut', 'Manchester', 6040, 4),
 ('Connecticut', 'Manchester', 6040, 4),
 ('Massachusetts', 'Franklin', 2038, 4),
 ('Rhode Island', 'Warwick', 2886, 4),
 ('Rhode Island', 'Warwick', 2886, 4),
 ('Massachusetts', 'Lawrence', 1841, 4),
 ('Massachusetts', 'Lawrence', 1841, 4),
 ('Massachusetts', 'Lowell', 1852, 4),
 ('Massachusetts', 'Lawrence', 1841, 4),
 ('

In [ ]:
#testing: cast it as TEXT to ensure it is string for any initial length (not a mixture of int and string)

result = cur.execute ('''SELECT
                            DISTINCT state,
                            city,
                            postal_code,
                            LENGTH(postal_code) AS code_length,
                            CASE
                              WHEN LENGTH(postal_code) = 4 THEN '0' || postal_code
                              WHEN LENGTH(postal_code) > 5 THEN SUBSTR(postal_code, 1, 5)
                              WHEN LENGTH(postal_code) < 4 THEN '99999' --error!
                              ELSE CAST(postal_code AS TEXT)
                          END AS cleaned_postal_code
                          FROM salesdb_table
                          WHERE state = 'Vermont' OR state = 'California' OR state = 'New York'
                          ORDER BY LENGTH(postal_code) DESC
                      ''')
result.fetchall()

[('California', 'Los Angeles', 90036, 5, '90036'),
 ('California', 'Los Angeles', 90032, 5, '90032'),
 ('California', 'San Francisco', 94109, 5, '94109'),
 ('California', 'Los Angeles', 90049, 5, '90049'),
 ('New York', 'New York City', 10024, 5, '10024'),
 ('New York', 'Troy', 12180, 5, '12180'),
 ('California', 'Los Angeles', 90004, 5, '90004'),
 ('New York', 'New York City', 10009, 5, '10009'),
 ('California', 'San Francisco', 94122, 5, '94122'),
 ('New York', 'New York City', 10035, 5, '10035'),
 ('California', 'Roseville', 95661, 5, '95661'),
 ('California', 'Pasadena', 91104, 5, '91104'),
 ('California', 'San Jose', 95123, 5, '95123'),
 ('California', 'Los Angeles', 90045, 5, '90045'),
 ('California', 'Redlands', 92374, 5, '92374'),
 ('California', 'Whittier', 90604, 5, '90604'),
 ('California', 'Santa Clara', 95051, 5, '95051'),
 ('California', 'San Diego', 92037, 5, '92037'),
 ('California', 'Brentwood', 94513, 5, '94513'),
 ('California', 'San Francisco', 94110, 5, '94110'),
 

In [ ]:
#make the sql more compact, load it into a new cleaned table
conn = sqlite3.connect("salesdb.sqlite") #had to start another conn, as I closed the other one too early
cur = conn.cursor()
cur.execute('DROP TABLE IF EXISTS cleaned_sales_table')

cur.execute('''
    CREATE TABLE cleaned_sales_table AS
    SELECT
            *
            , CASE
                WHEN LENGTH(postal_code) = 4 THEN '0' || postal_code
                WHEN LENGTH(postal_code) >= 5 THEN SUBSTR(postal_code, 1, 5)
                ELSE '99999' --error
                END AS cleaned_postal_code
    FROM salesdb_table
''')
#commit the changes to the database
conn.commit()



In [ ]:
res=cur.execute ('SELECT * FROM cleaned_sales_table')
res.fetchone()

(1,
 'CA-2016-152156',
 '11/8/2016',
 '11/11/2016',
 'Second Class',
 'CG-12520',
 'Claire Gute',
 'Consumer',
 'United States',
 'Henderson',
 'Kentucky',
 42420,
 'South',
 'FUR-BO-10001798',
 'Furniture',
 'Bookcases',
 'Bush Somerset Collection Bookcase',
 261.96,
 2,
 0.0,
 41.9136,
 '42420')

In [ ]:
#here is where I would connect Tableau desktop to the db, however with Tableau public there are limitations to what it can connect
#move the sql into a pandas df
cleaned_sales_df=pd.read_sql('SELECT * FROM cleaned_sales_table', conn)
cleaned_sales_df.head().transpose()

,0,1,2,3,4
row_id,1,2,3,4,5
order_id,CA-2016-152156,CA-2016-152156,CA-2016-138688,US-2015-108966,US-2015-108966
order_date,11/8/2016,11/8/2016,6/12/2016,10/11/2015,10/11/2015
ship_date,11/11/2016,11/11/2016,6/16/2016,10/18/2015,10/18/2015
ship_mode,Second Class,Second Class,Second Class,Standard Class,Standard Class
customer_id,CG-12520,CG-12520,DV-13045,SO-20335,SO-20335
customer_name,Claire Gute,Claire Gute,Darrin Van Huff,Sean O'Donnell,Sean O'Donnell
segment,Consumer,Consumer,Corporate,Consumer,Consumer
country,United States,United States,United States,United States,United States
city,Henderson,Henderson,Los Angeles,Fort Lauderdale,Fort Lauderdale


In [ ]:
cleaned_sales_df.info()
#notice that now we have 22 columns, including the cleaned_postal_code (now converted to string)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   row_id               9994 non-null   int64  
 1   order_id             9994 non-null   object 
 2   order_date           9994 non-null   object 
 3   ship_date            9994 non-null   object 
 4   ship_mode            9994 non-null   object 
 5   customer_id          9994 non-null   object 
 6   customer_name        9994 non-null   object 
 7   segment              9994 non-null   object 
 8   country              9994 non-null   object 
 9   city                 9994 non-null   object 
 10  state                9994 non-null   object 
 11  postal_code          9994 non-null   int64  
 12  region               9994 non-null   object 
 13  product_id           9994 non-null   object 
 14  category             9994 non-null   object 
 15  sub-category         9994 non-null   o

In [ ]:
cleaned_sales_df.head().transpose()

,0,1,2,3,4
row_id,1,2,3,4,5
order_id,CA-2016-152156,CA-2016-152156,CA-2016-138688,US-2015-108966,US-2015-108966
order_date,11/8/2016,11/8/2016,6/12/2016,10/11/2015,10/11/2015
ship_date,11/11/2016,11/11/2016,6/16/2016,10/18/2015,10/18/2015
ship_mode,Second Class,Second Class,Second Class,Standard Class,Standard Class
customer_id,CG-12520,CG-12520,DV-13045,SO-20335,SO-20335
customer_name,Claire Gute,Claire Gute,Darrin Van Huff,Sean O'Donnell,Sean O'Donnell
segment,Consumer,Consumer,Corporate,Consumer,Consumer
country,United States,United States,United States,United States,United States
city,Henderson,Henderson,Los Angeles,Fort Lauderdale,Fort Lauderdale


In [ ]:
#remove unneeded columns (perform data reduction):
print(cleaned_sales_df.columns.tolist())
#cleaned_sales_df.columns = cleaned_sales_df.drop(['postal_code', 'ship_date'], axis = 1)


['row_id', 'order_id', 'order_date', 'ship_date', 'ship_mode', 'customer_id', 'customer_name', 'segment', 'country', 'city', 'state', 'postal_code', 'region', 'product_id', 'category', 'sub-category', 'product_name', 'sales', 'quantity', 'discount', 'profit', 'cleaned_postal_code']


In [ ]:
#cleaned_sales_df = cleaned_sales_df.drop(['postal_code', 'ship_date'], axis = 1) #this did not work??
cols_to_drop = ['postal_code', 'ship_date']
existing_cols = [col for col in cols_to_drop if col in cleaned_sales_df.columns]
cleaned_sales_df = cleaned_sales_df.drop(existing_cols, axis=1)
cleaned_sales_df.head()

,row_id,order_id,order_date,ship_mode,customer_id,customer_name,segment,country,city,state,region,product_id,category,sub-category,product_name,sales,quantity,discount,profit,cleaned_postal_code
0,1,CA-2016-152156,11/8/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,42420
1,2,CA-2016-152156,11/8/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,42420
2,3,CA-2016-138688,6/12/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714,90036
3,4,US-2015-108966,10/11/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,33311
4,5,US-2015-108966,10/11/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164,33311


In [ ]:
#reorder columns so cleaned_postal_code is right after state like the original field
cols = cleaned_sales_df.columns.tolist()

# Remove 'cleaned_postal_code' from current position
cols.remove('cleaned_postal_code')

# Find index of 'state' and insert 'cleaned_postal_code' right after
state_index = cols.index('state')
cols.insert(state_index + 1, 'cleaned_postal_code')

# Reorder the DataFrame
cleaned_sales_df = cleaned_sales_df[cols]
cleaned_sales_df.head()

,row_id,order_id,order_date,ship_mode,customer_id,customer_name,segment,country,city,state,cleaned_postal_code,region,product_id,category,sub-category,product_name,sales,quantity,discount,profit
0,1,CA-2016-152156,11/8/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [ ]:
cleaned_sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   row_id               9994 non-null   int64  
 1   order_id             9994 non-null   object 
 2   order_date           9994 non-null   object 
 3   ship_mode            9994 non-null   object 
 4   customer_id          9994 non-null   object 
 5   customer_name        9994 non-null   object 
 6   segment              9994 non-null   object 
 7   country              9994 non-null   object 
 8   city                 9994 non-null   object 
 9   state                9994 non-null   object 
 10  cleaned_postal_code  9994 non-null   object 
 11  region               9994 non-null   object 
 12  product_id           9994 non-null   object 
 13  category             9994 non-null   object 
 14  sub-category         9994 non-null   object 
 15  product_name         9994 non-null   o

In [ ]:
cleaned_sales_df.describe()

,row_id,sales,quantity,discount,profit
count,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000
mean,4997.500000,229.858001,3.789574,0.156203,28.656896
std,2885.163629,623.245101,2.225110,0.206452,234.260108
min,1.000000,0.444000,1.000000,0.000000,-6599.978000
25%,2499.250000,17.280000,2.000000,0.000000,1.728750
50%,4997.500000,54.490000,3.000000,0.200000,8.666500
75%,7495.750000,209.940000,5.000000,0.200000,29.364000
max,9994.000000,22638.480000,14.000000,0.800000,8399.976000


In [ ]:
#will have to comment this out later:
cur.execute ("DROP TABLE IF EXISTS salesdb_table")
conn.close()

In [ ]:
#sql to data frame -> was done already earlier

#data frame to csv - file is saved in /content/sample_superstore_tr.csv, which is the working dir for Colab virtual machine, which is temporary.
cleaned_sales_df.to_csv('sample_superstore_tr.csv', index = False) # .csv extension needs to be included to prevent issue with opening file

#cannot write to github without authentification
#path = 'https://raw.githubusercontent.com/cristina0000/2025_git_projects/refs/heads/main/p5_tableau_superstore/data/'
#cleaned_sales_df.to_csv(path+'sample_superstore_tr.csv', index = False)




In [ ]:
#You can check the current directory for Colab with:
import os
print(os.getcwd())  # will show: /content

/content


In [ ]:
#check the file was saved to the working dir (temporay until the session ends)
for f in os.listdir():
    print(os.path.abspath(f))

/content/.config
/content/tutorialcsv.db
/content/salesdb.sqlite
/content/sample_superstore_tr.csv
/content/sales.sqlite
/content/sample_data


In [ ]:
#filter on csv files only and here is the output file:
[csv for csv in os.listdir() if csv.endswith('.csv')]

['sample_superstore_tr.csv']

In [ ]:
#check the output file was saved
import os

if 'sample_superstore_tr.csv' in os.listdir():
    print("✅ File saved:", os.path.abspath('sample_superstore_tr.csv'))
else:
    print("❌ File not found.")

✅ File saved: /content/sample_superstore_tr.csv


In [ ]:
#download file to local machine:
from google.colab import files
files.download("sample_superstore_tr.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>